In [1]:
from DrissionPage import ChromiumPage, ChromiumOptions
from urllib.parse import urlparse

In [2]:
website = ChromiumPage()

In [224]:
website.get("https://www.youtube.com/watch?v=wr5iRqSGe0A")

True

In [63]:
description = website.ele("@id=bottom-row").text

In [64]:
description

"86 тыс. просмотров5 месяцев назад\n 86 860 просмотров • 23 июл. 2025\u202fг. \nIn this video I tried faceless YouTube for 30 Days (realistic results). How much money can I make with faceless youtube in 30 days? I'm an ex-marketing agency owner with over 8 years of experience in social media content creation. I'm trying to make money online and do not sell any courses. Subscribe to get money-making ideas Timestamps: 0:00 - Intro 0:24 - Day 1 1:39 - Day 2 1:56 - Day 4 2:41 - Day 8 4:16 - Day 10 5:21 - Day 12 6:49 - Day 13 7:51 - Day 19 9:14 - Day 26 10:07 - Day 30 Business Inquiries: freddyvlogz3@gmail.com\nСвернуть \nIn this video I tried faceless YouTube for 30 Days (realistic results). How much money can I make with faceless youtube in 30 days? \n …\n...ещё \n...ещё \nЭпизоды\nПоказать все\nIntro\nIntro\n0:00\nIntro\n0:00\nDay 1\nDay 1\n0:24\nDay 1\n0:24\nDay 2\nDay 2\n1:39\nDay 2\n1:39\nDay 4\nDay 4\n1:56\nDay 4\n1:56\nDay 8\nDay 8\n2:41\nDay 8\n2:41\nDay 10\nDay 10\n4:16\nDay 10\n4

In [65]:
import re

def extract_emails(text):
    # Регулярное выражение для поиска стандартных email-адресов
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    
    # Находим все совпадения
    emails = re.findall(email_pattern, text)
    
    # Убираем дубликаты, если они есть
    return list(set(emails))


# Извлечение
found_emails = extract_emails(description)

# Вывод результата
if found_emails:
    print(f"Найдено почт: {len(found_emails)}")
    for email in found_emails:
        print(f"-> {email}")
else:
    print("Почта не найдена.")

Найдено почт: 1
-> freddyvlogz3@gmail.com


In [207]:
website.ele("@id=owner").eles("@class=style-scope ytd-watch-metadata")[0].click()

<ChromiumElement ytd-video-owner-renderer watch-metadata-refresh='' class='style-scope ytd-watch-metadata'>

In [209]:
website.ele("@class=ytDescriptionPreviewViewModelHost yt-page-header-view-model__page-header-description ytDescriptionPreviewViewModelClickable").click()

<ChromiumElement yt-description-preview-view-model class='ytDescriptionPreviewViewModelHost yt-page-header-view-model__page-header-description ytDescriptionPreviewViewModelClickable'>

In [106]:
def extract_instagram_links(text: str) -> list:
    pattern = r'(?:https?:\/\/)?(?:www\.)?instagram\.com\/[A-Za-z0-9._]+'
    return re.findall(pattern, text)

In [107]:
description = website.ele("@id=description-container").text

In [108]:
extract_instagram_links(description)

['https://www.instagram.com/freddy_henle']

In [182]:
info = website.ele("@id=additional-info-container").eles("tag:td@class=style-scope ytd-about-channel-renderer")

In [178]:
website.run_cdp("""
document.querySelectorAll('ytd-video-primary-info-renderer:not(.style-scope.ytd-video-primary-info-renderer)').forEach(el => el.remove());
""")

RuntimeError: 
没有找到对应功能，方法错误或你的浏览器太旧。
浏览器版本: Chrome/143.0.7499.171
方法: 
document.querySelectorAll('ytd-video-primary-info-renderer:not(.style-scope.ytd-video-primary-info-renderer)').forEach(el => el.remove());

版本: 4.1.0.18

In [174]:
for i in range(len(info)):
    print(info[i].text, i)

 0
Чтобы посмотреть адрес электронной почты, войдите в аккаунт. 1
 2
 3
 4
 5
 6
 7
 8
www.youtube.com/@Freddy-henle 9
 10
Соединенные Штаты 11
 12
Дата регистрации: 8 янв. 2017 г. 13
 14
42,6 тыс. подписчиков 15
 16
76 видео 17
 18
3 734 972 просмотра 19


In [181]:
parse_youtube_info(website.ele("@id=additional-info-container").text)

{'subscribers': 42600, 'videos': 76, 'views': 3734972}

In [171]:
def parse_number(text: str) -> int:
    """
    Преобразует:
    - '42,6 тыс. подписчиков' -> 42600
    - '3 734 972 просмотра' -> 3734972
    - '76 видео' -> 76
    """
    text = text.lower()

    # тыс.
    if 'тыс' in text:
        num = re.search(r'([\d,\.]+)', text)
        if num:
            return int(float(num.group(1).replace(',', '.')) * 1000)

    # млн (на будущее)
    if 'млн' in text:
        num = re.search(r'([\d,\.]+)', text)
        if num:
            return int(float(num.group(1).replace(',', '.')) * 1_000_000)

    # обычные числа с пробелами
    num = re.search(r'([\d\s]+)', text)
    if num:
        return int(num.group(1).replace(' ', ''))

    return 0

def extract_metrics(info):
    country = info[11].text.strip()
    channel_link = info[9].text.strip()

    subs = parse_number(info[15].text)
    videos = parse_number(info[17].text)
    views = parse_number(info[19].text)

    return {
        "country": country,
        "channel_link": channel_link,
        "subscribers": subs,
        "videos": videos,
        "views": views
    }


In [183]:
extract_metrics(info)

{'country': 'Соединенные Штаты',
 'channel_link': 'www.youtube.com/@ai_guy',
 'subscribers': 277000,
 'videos': 27,
 'views': 2786302}

In [216]:
website.eles("@class=style-scope ytd-video-renderer")

[<ChromiumElement div id='dismissible' class='style-scope ytd-video-renderer'>,
 <ChromiumElement ytd-thumbnail use-hovered-property='' class='style-scope ytd-video-renderer' size='large' loaded=''>,
 <ChromiumElement div id='meta' class='style-scope ytd-video-renderer'>,
 <ChromiumElement div id='title-wrapper' class='style-scope ytd-video-renderer'>,
 <ChromiumElement ytd-badge-supported-renderer collection-truncate='' class='style-scope ytd-video-renderer' system-icons='' use-badge-shape='' hidden=''>,
 <ChromiumElement yt-icon id='inline-title-icon' class='style-scope ytd-video-renderer' hidden=''>,
 <ChromiumElement yt-formatted-string class='style-scope ytd-video-renderer' aria-label='How to Start a Faceless YouTube Channel with AI 12 минут 54 секунды'>,
 <ChromiumElement div id='menu' class='style-scope ytd-video-renderer'>,
 <ChromiumElement ytd-menu-renderer class='style-scope ytd-video-renderer' safe-area='' menu-active=''>,
 <ChromiumElement ytd-video-meta-block class='style

In [229]:
website.eles("tag:a@id=thumbnail")

[<ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=HgOma-23SXQ&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=yPuxoqgovto&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/shorts/IrAbQ4udcDg'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/shorts/P9URaD1Fo8o'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/shorts/Vc2O_BaiXNs'>,
 <ChromiumElement a id='thumbnai

In [ ]:
for i in lst:
        contact_info = ""
        country = ""
        channel_link = ""
        subscribers = 0
        videos = 0
        views = 0
        website.scroll.to_see(i)
        i.click()
        website.refresh()
        random_sleep(5,10)
        info = website.ele("@class=style-scope ytd-video-renderer")
        description = website.ele("@id=bottom-row").text
        contact_info = extract_emails(description)
        if not contact_info:
            contact_info = extract_instagram_links(description)
        
        website.ele("@id=owner").eles("@class=style-scope ytd-watch-metadata")[0].click()
        website.refresh()
        random_sleep(5,10)
        website.ele("@class=ytDescriptionPreviewViewModelHost yt-page-header-view-model__page-header-description ytDescriptionPreviewViewModelClickable").click()
        if not contact_info:
            contact_info = extract_instagram_links(website.ele("@id=description-container").text)
        
        info = website.ele("@id=additional-info-container").eles("tag:td@class=style-scope ytd-about-channel-renderer")
        country = info[11].text.strip()
        channel_link = info[9].text.strip()
        subscribers = parse_number(info[15].text)
        videos = parse_number(info[17].text)
        views = parse_number(info[19].text)
        website.back()
        random_sleep(1,3)
        website.back()
        random_sleep(1,3)
        infos[channel_link] = {
            "contact_info": contact_info,
            "country": country,
            "subscribers": subscribers,
            "videos": videos,
            "views": views
        }
    print(infos)

In [240]:
lst = website.eles("tag:a@id=thumbnail")
lst

[<ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=HgOma-23SXQ&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=pXPu3hAR2zE&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=EyXNKb2HsfM&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hidden='true' tabindex='-1' rel='null' href='/watch?v=yPuxoqgovto&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D'>,
 <ChromiumElement a id='thumbnail' class='yt-simple-endpoint inline-block style-scope ytd-thumbnail' aria-hi

In [241]:
links = []
for i in lst:
    attr = i.attr("href")
    if attr:
        links.append(attr)

In [242]:
links

['https://www.youtube.com/watch?v=HgOma-23SXQ&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=pXPu3hAR2zE&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=EyXNKb2HsfM&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=yPuxoqgovto&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/shorts/Vc2O_BaiXNs',
 'https://www.youtube.com/watch?v=OTLxMmrMyYo&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=53Vi-gNyGoU&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbNIHCQlNCgGHKiGM7w%3D%3D',
 'https://www.youtube.com/watch?v=lqXWVzWkkyc&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=Qucq3h_iL1s&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/watch?v=oYrFyTYc6dg&pp=ygUZZmFjZWxlc3MgeW91dHViZSB0dXRvcmlhbA%3D%3D',
 'https://www.youtube.com/shorts/M4yB_k4qp70',
 'https://www.youtube.com/

In [245]:
website.url.split("/")

['https:', '', 'www.youtube.com', 'shorts', 'er0jkAQEYU8']

In [246]:
website.ele("@class=ytReelMultiFormatLinkViewModelEndpoint").attr("href")

'https://www.youtube.com/watch?v=DAnPdNEscF8'

In [255]:
import time
website.get("https://www.youtube.com/shorts/M4yB_k4qp70")
time.sleep(1)
if website.url.split("/")[3] == "shorts":
    website.get(website.ele("@class=ytReelMultiFormatLinkViewModelEndpoint").attr("href"))
    time.sleep(1)
    website.refresh()
    print("lol")

ElementNotFoundError: 
没有找到元素。
方法: ele()
参数: {'locator': '@class=ytReelMultiFormatLinkViewModelEndpoint', 'index': 1, 'timeout': 10}
版本: 4.1.0.18

In [257]:
website.ele("tag:h2").click()

<ChromiumElement h2 class='ytShortsVideoTitleViewModelShortsVideoTitle'>

In [265]:
website.ele("@id=owner").eles("@class=style-scope ytd-watch-metadata")[0].click()

<ChromiumElement ytd-video-owner-renderer watch-metadata-refresh='' class='style-scope ytd-watch-metadata'>

In [271]:
website.ele("@class=style-scope ytd-channel-name complex-string").click()

<ChromiumElement yt-formatted-string id='text' link-inherit-color='' respect-lang-dir='' title='vidIQ' class='style-scope ytd-channel-name complex-string' ellipsis-truncate='' ellipsis-truncate-styling='' dir='auto' style='text-align: left;' has-link-only_=''>

In [272]:
infos = dict()
infos["111"] = {"o":1, "1":[1,2,3]}

AttributeError: 'ChromiumPage' object has no attribute 'headless'

In [275]:
import pandas as pd

In [276]:
df = pd.read_excel("output.xlsx")
df.to_csv("data.csv", index=False)

In [ ]:
co = ChromiumOptions()
co.headless()
website = ChromiumPage()

In [6]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    # Убираем порт отсюда, оставляем только домен
    host='s8uvo69uiv.eu-west-2.aws.clickhouse.cloud', 
    port=8443,
    username='default',
    password='K_OeqymQxv8qB',
    secure=True
)

# Тестовый запрос
result = client.query('SELECT version()')
print(f"Успех! Версия ClickHouse: {result.result_rows[0][0]}")

Успех! Версия ClickHouse: 25.8.1.8909


In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("data.csv")

In [9]:
df

,www.youtube.com/@ai_with_gary,www.youtube.com/@Yourivanhofwegen,www.youtube.com/@razvanparaschiv,www.youtube.com/@JoeSappy,www.youtube.com/@BigStepsMedia,www.youtube.com/@vidIQ,www.youtube.com/@syraxverse,www.youtube.com/@OnAMissionToTheLoot,www.youtube.com/@ai_makemoneytools,www.youtube.com/@ai_guy,...,www.youtube.com/@heygen_official,www.youtube.com/@KenVossAI,www.youtube.com/@Vivaldi_ai,www.youtube.com/@Geslene,www.youtube.com/@ai-automation-station,www.youtube.com/@AIFromScratch-q2k,www.youtube.com/@Rebelvids_01,www.youtube.com/@editspro94,www.youtube.com/@Bye9To5Official,www.youtube.com/@createyourinfluence
0,[],['youribrands@gmail.com'],['instagram.com/razvanparaschiv'],[],[],['support@vidIQ.com'],[],[],[],[],...,[],['kenvossbusiness@gmail.com'],[],['growtowealthgm@gmail.com'],[],[],[],[],['support@bye9to5official.com'],['https://www.instagram.com/createyourinfluence']
1,Соединенные Штаты,Великобритания,Соединенные Штаты,NaN,Соединенные Штаты,Соединенные Штаты,Соединенные Штаты,Канада,Румыния,Соединенные Штаты,...,Соединенные Штаты,Соединенные Штаты,Италия,Соединенные Штаты,Таиланд,Соединенные Штаты,NaN,Германия,Соединенные Штаты,Соединенные Штаты
2,27700,245000,60300,7730,50100,2310000,34900,29500,3180,277000,...,72200,105000,615,13200,2150,906,97700,29400,50700,15300
3,48,169,94,51,68,2459,45,1,125,27,...,596,54,15,55,117,69,90,31,125,173
4,699992,11587597,1291713,315850,2153687,242179649,1700263,1121693,312802,2787179,...,36573369,147247,22636,510060,110035,44437,7997256,1953656,1370061,1049602


In [10]:
df_transformed = df.T.reset_index()

# 3. Назначаем правильные имена столбцов
# В твоем CSV строки шли в таком порядке: 0-контакты, 1-страна, 2-сабы, 3-видео, 4-просмотры
df_transformed.columns = [
    'creators', 
    'contact_info', 
    'country', 
    'total_subs', 
    'total_vids', 
    'total_views'
]

# 4. Чистим данные
# Убираем квадратные скобки и кавычки из контактов
df_transformed['contact_info'] = df_transformed['contact_info'].str.replace(r"[\[\]']", "", regex=True)

# Превращаем цифры из строк в реальные числа (чтобы ClickHouse не ругался)
cols_to_fix = ['total_subs', 'total_vids', 'total_views']
for col in cols_to_fix:
    df_transformed[col] = pd.to_numeric(df_transformed[col], errors='coerce').fillna(0).astype(int)

# 5. Упорядочиваем столбцы так, как ты просил
final_columns = ['creators', 'country', 'total_subs', 'total_views', 'contact_info', 'total_vids']
df_final = df_transformed[final_columns]


In [12]:
df_transformed.to_csv("final.csv")

In [13]:
client.insert_df(
    table='youtube_creators',
    df=df_transformed,
    database='default'
)

In [14]:
row_count = client.query('SELECT count() FROM default.youtube_creators').result_rows[0][0]
print(f"Готово! Теперь в таблице {row_count} строк.")

Готово! Теперь в таблице 111 строк.
